In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [188]:
def Generalized_eigenvector(A,B):
  eig_mat_B = np.linalg.eig(B)[1] #### (v_{B,1}, ...., v_{B,p})
  eig_mat_B_tild = np.dot(eig_mat_B, np.diag(np.linalg.eig(B)[0]**(-0.5)))
  A_tild = np.dot(eig_mat_B_tild.T, A).dot(eig_mat_B_tild)
  eig_mat_A = np.linalg.eig(A_tild)[1]
  eig_mat_A_val = np.linalg.eig(A_tild)[0]

  res = np.dot(eig_mat_B_tild, eig_mat_A)
  sorted_indices = np.argsort(eig_mat_A_val)[::-1]
  eig_mat_A_val = eig_mat_A_val[sorted_indices]
  res = res[:, sorted_indices]
  return res

def Generalized_eigenvector_topk_2(A,B):
  s = 10
  eig_mat_B = np.linalg.eig(B)[1][:,:10] #### (v_{B,1}, ...., v_{B,p})
  eig_mat_B_tild = np.dot(eig_mat_B, np.diag(np.linalg.eig(B)[0][:10]**(-0.5)))
  A_tild = np.dot(eig_mat_B_tild.T, A).dot(eig_mat_B_tild)
  A_tild = A_tild + np.eye(len(A_tild))*0
  eig_mat_A = np.linalg.eig(A_tild)[1]
  eig_mat_A_val = np.linalg.eig(A_tild)[0]

  res = np.dot(eig_mat_B, eig_mat_A)
  sorted_indices = np.argsort(eig_mat_A_val)[::-1]
  eig_mat_A_val = eig_mat_A_val[sorted_indices]
  res = res[:, sorted_indices]
  return res

def SinTheta(U,W):  #### U \in R^{p*k}
  tmp = np.matmul(U.T, W)
  _, S, _ = np.linalg.svd(tmp, full_matrices=True)
  return np.sin(np.arccos(min(S)))

def TheoryVal(asp_rat, min_sig):
  res = 1 - (1 - asp_rat/min_sig**2)/(1 + asp_rat/min_sig)
  return np.sqrt(res)

def generate_data_2(p = 400, n = 1000, sig_lev = [1,2,3,4,5], bkg_lev = [1,2,3,4,5]):

  A = np.zeros((p, 5))
  A[0][0] = np.sqrt(sig_lev[0])
  A[1][1] = np.sqrt(sig_lev[1])
  A[2][2] = np.sqrt(sig_lev[2])
  A[3][3] = np.sqrt(sig_lev[3])
  A[4][4] = np.sqrt(sig_lev[4])

  B = np.zeros((p, 5))
  B[-1][0] = np.sqrt(bkg_lev[0])
  B[-2][1] = np.sqrt(bkg_lev[1])
  B[-3][2] = np.sqrt(bkg_lev[2])
  B[-4][3] = np.sqrt(bkg_lev[3])
  B[-5][4] = np.sqrt(bkg_lev[4])


  epsilon = np.random.normal(0,1, size = (n,p))
  epsilon_2 = np.random.normal(0,1, size = (n,p))

  F = np.random.normal(0,1, size = (n,5))

  L = np.random.normal(0,1, size = (n,5))
  Lplus = np.random.normal(0,1, size = (n,5))

  X = F.dot(A.T) + L.dot(B.T) + epsilon
  Xplus = F.dot(A.T) + Lplus.dot(B.T) + epsilon_2

  S = X.T.dot(X)/n
  Splus = (X.T.dot(Xplus) + Xplus.T.dot(X))/2/n
  return S, Splus

In [307]:
n = 100
p = 40
N = 50

res = [[0]*N for _ in range(4)]

sig_lev = [10,10,15,20,20]
bkg_lev = [100,100,200,500,500]

#### True space
Sig = np.zeros((p, 5))
Sig[0][0] = 1
Sig[1][1] = 1
Sig[2][2] = 1
Sig[3][3] = 1
Sig[4][4] = 1

Bkg = np.zeros((p, 5))
Bkg[-1][0] = 1
Bkg[-2][1] = 1
Bkg[-3][2] = 1
Bkg[-4][3] = 1
Bkg[-5][4] = 1

for i in range(N):
  S, Splus= generate_data_2(p = p, n = n, sig_lev = sig_lev, bkg_lev = bkg_lev)

  vec = np.linalg.eig(S)[1].T[:5]
  vec_cl = np.linalg.eig(Splus)[1].T[:5]
  vec_unif_1 = Generalized_eigenvector_topk_2(Splus,S).T[:5]

  res[0][i] = SinTheta(vec.T, Sig)
  res[1][i] = SinTheta(vec_cl.T, Sig)
  res[2][i] = SinTheta(vec_unif_1.T, Sig)
  res[3][i] = TheoryVal(p/n, min(sig_lev))

m = [0]*4
sd = [0]*4
for i in range(4):
  m[i] = float(np.mean(res[i]))
  sd[i] = float(np.std(res[i]))
print(m,sd)

[0.9999492032812097, 0.9993809736118378, 0.3048449487899472, 0.2056883378018608] [6.499138116543227e-05, 0.0008478774571379029, 0.037254140906155066, 5.551115123125783e-17]


In [308]:
n = 500
p = 200
N = 50

res = [[0]*N for _ in range(4)]

sig_lev = [10,10,15,20,20]
bkg_lev = [100,100,200,500,500]

#### True space
Sig = np.zeros((p, 5))
Sig[0][0] = 1
Sig[1][1] = 1
Sig[2][2] = 1
Sig[3][3] = 1
Sig[4][4] = 1

Bkg = np.zeros((p, 5))
Bkg[-1][0] = 1
Bkg[-2][1] = 1
Bkg[-3][2] = 1
Bkg[-4][3] = 1
Bkg[-5][4] = 1

for i in range(N):
  S, Splus= generate_data_2(p = p, n = n, sig_lev = sig_lev, bkg_lev = bkg_lev)

  vec = np.linalg.eig(S)[1].T[:5]
  vec_cl = np.linalg.eig(Splus)[1].T[:5]
  vec_unif_1 = Generalized_eigenvector_topk_2(Splus,S).T[:5]

  res[0][i] = SinTheta(vec.T, Sig)
  res[1][i] = SinTheta(vec_cl.T, Sig)
  res[2][i] = SinTheta(vec_unif_1.T, Sig)
  res[3][i] = TheoryVal(p/n, min(sig_lev))

m = [0]*4
sd = [0]*4
for i in range(4):
  m[i] = float(np.mean(res[i]))
  sd[i] = float(np.std(res[i]))
print(m,sd)

[0.9999929165611983, 0.9996295506077747, 0.2258428234000012, 0.2056883378018608] [1.0597335337004228e-05, 0.0006860107496815607, 0.010850011281416945, 5.551115123125783e-17]


In [309]:
n = 2500
p = 1000
N = 50

res = [[0]*N for _ in range(4)]

sig_lev = [10,10,15,20,20]
bkg_lev = [100,100,200,500,500]

#### True space
Sig = np.zeros((p, 5))
Sig[0][0] = 1
Sig[1][1] = 1
Sig[2][2] = 1
Sig[3][3] = 1
Sig[4][4] = 1

Bkg = np.zeros((p, 5))
Bkg[-1][0] = 1
Bkg[-2][1] = 1
Bkg[-3][2] = 1
Bkg[-4][3] = 1
Bkg[-5][4] = 1

for i in range(N):
  S, Splus= generate_data_2(p = p, n = n, sig_lev = sig_lev, bkg_lev = bkg_lev)

  vec = np.linalg.eig(S)[1].T[:5]
  vec_cl = np.linalg.eig(Splus)[1].T[:5]
  vec_unif_1 = Generalized_eigenvector_topk_2(Splus,S).T[:5]

  res[0][i] = SinTheta(vec.T, Sig)
  res[1][i] = SinTheta(vec_cl.T, Sig)
  res[2][i] = SinTheta(vec_unif_1.T, Sig)
  res[3][i] = TheoryVal(p/n, min(sig_lev))

m = [0]*4
sd = [0]*4
for i in range(4):
  m[i] = float(np.mean(res[i]))
  sd[i] = float(np.std(res[i]))
print(m,sd)

[0.9999982030501614, 0.9994580725264504, 0.21297995941066275, 0.2056883378018608] [2.3839402387740647e-06, 0.001259469360157635, 0.0050892073651034285, 5.551115123125783e-17]


In [310]:
n = 100
p = 40
N = 50

res = [[0]*N for _ in range(4)]

sig_lev = [10,10,15,20,20]
bkg_lev = [25,25,50,100,100]

#### True space
Sig = np.zeros((p, 5))
Sig[0][0] = 1
Sig[1][1] = 1
Sig[2][2] = 1
Sig[3][3] = 1
Sig[4][4] = 1

Bkg = np.zeros((p, 5))
Bkg[-1][0] = 1
Bkg[-2][1] = 1
Bkg[-3][2] = 1
Bkg[-4][3] = 1
Bkg[-5][4] = 1

for i in range(N):
  S, Splus= generate_data_2(p = p, n = n, sig_lev = sig_lev, bkg_lev = bkg_lev)

  vec = np.linalg.eig(S)[1].T[:5]
  vec_cl = np.linalg.eig(Splus)[1].T[:5]
  vec_unif_1 = Generalized_eigenvector_topk_2(Splus,S).T[:5]

  res[0][i] = SinTheta(vec.T, Sig)
  res[1][i] = SinTheta(vec_cl.T, Sig)
  res[2][i] = SinTheta(vec_unif_1.T, Sig)
  res[3][i] = TheoryVal(p/n, min(sig_lev))

m = [0]*4
sd = [0]*4
for i in range(4):
  m[i] = float(np.mean(res[i]))
  sd[i] = float(np.std(res[i]))
print(m,sd)

[0.9997542776001838, 0.9960281560487774, 0.29338264346148074, 0.2056883378018608] [0.0003731406801557514, 0.01163141514259555, 0.038515480984905245, 5.551115123125783e-17]


In [311]:
n = 500
p = 200
N = 50

res = [[0]*N for _ in range(4)]

sig_lev = [10,10,15,20,20]
bkg_lev = [25,25,50,100,100]

#### True space
Sig = np.zeros((p, 5))
Sig[0][0] = 1
Sig[1][1] = 1
Sig[2][2] = 1
Sig[3][3] = 1
Sig[4][4] = 1

Bkg = np.zeros((p, 5))
Bkg[-1][0] = 1
Bkg[-2][1] = 1
Bkg[-3][2] = 1
Bkg[-4][3] = 1
Bkg[-5][4] = 1

for i in range(N):
  S, Splus= generate_data_2(p = p, n = n, sig_lev = sig_lev, bkg_lev = bkg_lev)

  vec = np.linalg.eig(S)[1].T[:5]
  vec_cl = np.linalg.eig(Splus)[1].T[:5]
  vec_unif_1 = Generalized_eigenvector_topk_2(Splus,S).T[:5]

  res[0][i] = SinTheta(vec.T, Sig)
  res[1][i] = SinTheta(vec_cl.T, Sig)
  res[2][i] = SinTheta(vec_unif_1.T, Sig)
  res[3][i] = TheoryVal(p/n, min(sig_lev))

m = [0]*4
sd = [0]*4
for i in range(4):
  m[i] = float(np.mean(res[i]))
  sd[i] = float(np.std(res[i]))
print(m,sd)

[0.9999428638906243, 0.7138123771932324, 0.22770310884883418, 0.2056883378018608] [7.592506452524923e-05, 0.28790693015287866, 0.0102031881374062, 5.551115123125783e-17]


In [312]:
n = 2500
p = 1000
N = 50

res = [[0]*N for _ in range(4)]

sig_lev = [10,10,15,20,20]
bkg_lev = [25,25,50,100,100]

#### True space
Sig = np.zeros((p, 5))
Sig[0][0] = 1
Sig[1][1] = 1
Sig[2][2] = 1
Sig[3][3] = 1
Sig[4][4] = 1

Bkg = np.zeros((p, 5))
Bkg[-1][0] = 1
Bkg[-2][1] = 1
Bkg[-3][2] = 1
Bkg[-4][3] = 1
Bkg[-5][4] = 1

for i in range(N):
  S, Splus= generate_data_2(p = p, n = n, sig_lev = sig_lev, bkg_lev = bkg_lev)

  vec = np.linalg.eig(S)[1].T[:5]
  vec_cl = np.linalg.eig(Splus)[1].T[:5]
  vec_unif_1 = Generalized_eigenvector_topk_2(Splus,S).T[:5]

  res[0][i] = SinTheta(vec.T, Sig)
  res[1][i] = SinTheta(vec_cl.T, Sig)
  res[2][i] = SinTheta(vec_unif_1.T, Sig)
  res[3][i] = TheoryVal(p/n, min(sig_lev))

m = [0]*4
sd = [0]*4
for i in range(4):
  m[i] = float(np.mean(res[i]))
  sd[i] = float(np.std(res[i]))
print(m,sd)

[0.9999842768422705, 0.22296567220510144, 0.21219516972264452, 0.2056883378018608] [1.9604630035003488e-05, 0.06271554730158277, 0.004266133459020698, 5.551115123125783e-17]
